In [1]:
import pandas as pd
from nltk.stem import PorterStemmer
import time
import sys
import ast
from pickle import load
import json 

ps = PorterStemmer() # stemming for better results 

In [2]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [6]:
def search(grocery_list, ps):
    stores = ['zehrs', 'no_frills', 'valu_mart', 'sobeys', 'freshco'] #, 'walmart', 'food_basics']

    # make variables 
    for store in stores:
        globals()[f"{store}_results"] = pd.DataFrame() # dynamically create variable names 

        
    for store in stores: # retrieval for each store 
        
        # load data
        store_data = pd.read_csv(f'clean_data/{store}/{store}_data.csv')

        # load index
        globals()[f"{store}_index"] = load(open(f"catalogue_index/{store}_index.pkl",'rb'))
        
        final_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'is_sale'])
        
        for item in grocery_list:
            item_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'is_sale'])

            stem_item = ps.stem(item).lower()
            idxs = []

            for word in stem_item.split():
                try: # list item word not in index
                    word_idxs = globals()[f"{store}_index"][word]
                    idxs.extend(word_idxs)
                except: continue

            if not idxs: # no indicies returned
                store_df = pd.DataFrame() # no results: return empty df
            else:
                store_df = store_data.iloc[idxs]


            ##### search items #####
            for index, row in store_df.iterrows():
                
                product_name = row['product']
                is_sale = row.is_sale

                if is_sale:
                    price = row.sale_price
                    per_unit_price = row.sale_per_unit_price
                else:
                    price = row.price
                    per_unit_price = row.per_unit_price

                similarity = jaccard_similarity(stem_item.split(' '), ps.stem(product_name).lower().split(' '))
        
                if similarity >= 0.5: # can tweak threshold but this is a good one for now  
                    data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price':per_unit_price, 'similarity':similarity, 'is_sale': is_sale}
                    item_selection = item_selection.append(data, ignore_index=True)    
           
            try:
                # find lowest price from top similarities
                cheapest_item = item_selection.sort_values(by=['per_unit_price', 'similarity'], ascending = [True, False])

                final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)
            except: continue
                
        globals()[f"{store}_results"] = final_selection
    
        # dump results to csv 
        globals()[f"{store}_results"].to_csv(f'search_output/{store}_results.csv')
    
    return {'zehrs': zehrs_results
        , 'no_frills': no_frills_results
        , 'valu_mart': valu_mart_results
        # , 'walmart': walmart_results
        , 'sobeys': sobeys_results
        , 'freshco': freshco_results}
        # , 'food_basics': food_basics_results}

In [7]:
def find_n_cheapest_stores(n, results):
    per_unit_subtotals = {}
    subtotals = {}
    cheapest_stores = {}

    for store in results.keys():
        result = results[store]
        per_unit_subtotal = round(result.per_unit_price.sum(),3)
        subtotal = round(result.price.sum(),3)

        per_unit_subtotals[store] = per_unit_subtotal
        subtotals[store] = subtotal

    for i in range(n):
        min_store = min(per_unit_subtotals, key=per_unit_subtotals.get)
        
        cost = subtotals[min_store]

        cheapest_stores[i+1] = {'store':store
                            , 'file_name': f'{store}_results.csv'
                            , 'subtotal': cost}

        remove_key = per_unit_subtotals.pop(min_store)
    
    return cheapest_stores

In [8]:
grocery_list = ['2% milk', 'Cheddar Cheese', 'white sliced bread', 'ground beef', 'clementines', 'chicken breast', 'potatoes']

results_dict = search(grocery_list, ps)

In [9]:
results_dict

{'zehrs':             list_item  store           product_name  price  per_unit_price  \
 0             2% milk  zehrs                2% Milk   4.99            0.25   
 1      Cheddar Cheese  zehrs  Medium Cheddar Cheese   9.79            1.40   
 2  white sliced bread  zehrs            White Bread   2.69            0.40   
 3         ground beef  zehrs       Lean Ground Beef  10.00            1.10   
 4         clementines  zehrs    Organic Clementines   9.99            1.10   
 5      chicken breast  zehrs  Chicken Breast Strips  13.99            1.75   
 6            potatoes  zehrs           Red Potatoes   5.99            0.26   
 
   is_sale  similarity  
 0   False    1.000000  
 1   False    0.666667  
 2   False    0.666667  
 3   False    0.666667  
 4   False    0.500000  
 5   False    0.666667  
 6   False    0.500000  ,
 'no_frills':             list_item      store                 product_name  price  \
 0             2% milk  no_frills          True Taste, 2% Milk   5.49 

# 2 store combination 

In [82]:
from itertools import combinations

In [83]:
list(combinations(results_dict.keys(), 2))

[('zehrs', 'no_frills'),
 ('zehrs', 'valu_mart'),
 ('zehrs', 'sobeys'),
 ('zehrs', 'freshco'),
 ('no_frills', 'valu_mart'),
 ('no_frills', 'sobeys'),
 ('no_frills', 'freshco'),
 ('valu_mart', 'sobeys'),
 ('valu_mart', 'freshco'),
 ('sobeys', 'freshco')]

In [84]:
out = pd.data_frame()

possibilities = list(combinations(results_dict.keys(), 2))
for combs in possibilities:
    for item in grocery_list: 
        df1 = pd.read_csv('search_output/valu_mart_results.csv')
        df2 = pd.read_csv('search_output/freshco_results.csv')

        

In [87]:
def choose_lowest(df1, df2, column_name):
    # Create a new DataFrame with the minimum value of the column for each row
    df = pd.DataFrame(columns=[column_name])
    df['list_item'] = df1.list_item
    df[column_name] = df1[column_name].combine(df2[column_name], min)
    return df

df1 = pd.read_csv('search_output/valu_mart_results.csv')
df2 = pd.read_csv('search_output/freshco_results.csv')

choose_lowest(df1, df2, 'comparable_price')

,comparable_price,list_item
0,0.309254,2% milk
1,1.400000,Cheddar Cheese
2,0.505070,white sliced bread
3,1.392842,ground beef
4,0.390000,clementines


In [74]:
df1[['list_item', 'comparable_price']]

,list_item,comparable_price
0,2% milk,0.35
1,Cheddar Cheese,1.40
2,white sliced bread,0.56
3,ground beef,15.41
4,clementines,0.39


In [75]:
df2[['list_item', 'comparable_price']]

,list_item,comparable_price
0,2% milk,0.309254
1,Cheddar Cheese,1.955578
2,white sliced bread,0.505070
3,ground beef,1.392842
4,chicken breast,1.741052


In [ ]:
import itertools

def lowest_price_multiple_df(df_list, column_name):
    results = []
    for df1, df2 in itertools.combinations(df_list, 2):
        result = lowest_price(df1, df2, column_name)
        results.append(result)
    return results

df1 = pd.DataFrame({'per_unit_price': [1, 2, 3, 4]})
df2 = pd.DataFrame({'per_unit_price': [4, 3, 2, 1]})
df3 = pd.DataFrame({'per_unit_price': [2, 3, 4, 5]})
df_list = [df1, df2, df3]
result_df = lowest_price_multiple_df(df_list, 'per_unit_price')


In [89]:
df2.head(1)

,store,category,brand,product,price,sale_price,per_unit_price,sale_per_unit_price,units,is_sale,list_item,similarity,comparable_price
0,freshco,dairy_and_eggs,Neilson,2% Milk,3.083447,NaN,0.309254,NaN,100ml,0.0,2% milk,100.0,0.309254


In [99]:
df1 = pd.read_csv('search_output/valu_mart_results.csv')
df2 = pd.read_csv('search_output/freshco_results.csv')

df = df1.append(df2, ignore_index=True)

dfc = df.groupby('list_item')['comparable_price']

df = df.assign(min_cost=dfc.transform(min))

df = df[df['comparable_price'] == df['min_cost']]

df

,Unnamed: 0,store,category,brand,product,price,sale_price,per_unit_price,sale_per_unit_price,units,is_sale,list_item,similarity,comparable_price,min_cost
1,24.0,valu_mart,dairy_and_eggs,No Name,Old Cheddar Cheese,9.790000,NaN,1.400000,NaN,100g,0.0,Cheddar Cheese,66.666667,1.400000,1.400000
4,1432.0,valu_mart,produce,NaN,Clementines,6.990000,NaN,0.390000,NaN,100g,0.0,clementines,100.000000,0.390000,0.390000
5,NaN,freshco,dairy_and_eggs,Neilson,2% Milk,3.083447,NaN,0.309254,NaN,100ml,0.0,2% milk,100.000000,0.309254,0.309254
7,NaN,freshco,bakery,Wonder,"Bread, Sliced White",3.418239,NaN,0.505070,NaN,100g,0.0,white sliced bread,75.000000,0.505070,0.505070
8,NaN,freshco,meat,NaN,Lean Ground Beef,6.964210,NaN,1.392842,NaN,100g,0.0,ground beef,66.666667,1.392842,1.392842
9,NaN,freshco,meat,Zabiha Halal,Chicken Breast Strips,13.918471,NaN,1.741052,NaN,100g,0.0,chicken breast,66.666667,1.741052,1.741052


In [37]:
from nltk.metrics.distance import jaccard_distance

jaccard_distance(set('Natrel 2% milk 4 L'.lower().split(' ')), set('2% milk'.lower().split(' ')))


0.6

In [21]:
set('Natrel 2% milk 4 L'.lower().split(' '))

{'2%', '4', 'l', 'milk', 'natrel'}

In [35]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

jaccard_similarity('Natrel 2% milk 4 L'.lower().split(' '), '2% milk'.lower().split(' '))

0.4

In [24]:
! pip install fuzzywuzzy

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [41]:
def jaccard_similarity(string1, string2):
    similarity = jaccard_distance(set(string1.lower()), set(string2.lower()))
    return int(similarity * 100)

jaccard_similarity('natrel 2% milk 4 L', 'milk 2%')

46

In [62]:
from fuzzywuzzy import process, fuzz
# import fuzzywuzzy as fuzz
import pandas as pd
from nltk.stem import PorterStemmer
ps = PorterStemmer()

df = pd.read_csv('clean_data/zehrs/zehrs_data.csv')

def jaccard_similarity(string1, string2):
    return (1 - jaccard_distance(set(string1.lower().split(' ')), set(string2.lower().split(' ')))) * 100


# Define the string to compare against
query = "2% milk"

# Use the process.extract() function to find the most similar items
results = process.extract(query, df['product'], scorer=jaccard_similarity, limit=30)

idxs = []
for item in results: 
    if item[1] >= 52: 
        idxs.append(item[2])

df = df.iloc[idxs]

import numpy as np 

df["comparable_price"] = np.where(df["is_sale"] == True, df["sale_price"], df["price"])

df


,Unnamed: 0,store,category,brand,product,price,sale_price,per_unit_price,sale_per_unit_price,units,is_sale,comparable_price
283,283,zehrs,dairy_and_eggs,Neilson,2% Milk,3.39,NaN,0.34,NaN,100ml,False,3.39
347,347,zehrs,dairy_and_eggs,Neilson,2% Milk,2.09,NaN,0.88,NaN,100ml,False,2.09
394,394,zehrs,dairy_and_eggs,Neilson,2% Milk,4.99,NaN,0.25,NaN,100ml,False,4.99
474,474,zehrs,dairy_and_eggs,Neilson,2% Milk,2.59,NaN,0.55,NaN,100ml,False,2.59
258,258,zehrs,dairy_and_eggs,Neilson,TruTaste 2% Milk,4.09,NaN,0.41,NaN,100ml,False,4.09
553,553,zehrs,dairy_and_eggs,PC Organics,Organic 2% Milk,4.59,NaN,0.46,NaN,100ml,False,4.59
68,68,zehrs,dairy_and_eggs,Rolling Meadow,Grass Fed 2% Milk,6.29,NaN,0.31,NaN,100ml,False,6.29
934,934,zehrs,dairy_and_eggs,Neilson,"True Taste, 2% Milk",4.99,NaN,0.25,NaN,100ml,False,4.99


In [51]:
results

[('2% Milk', 100.0, 283),
 ('2% Milk', 100.0, 347),
 ('2% Milk', 100.0, 394),
 ('2% Milk', 100.0, 474),
 ('TruTaste 2% Milk', 75.0, 258),
 ('Organic 2% Milk', 75.0, 553),
 ('Grass Fed 2% Milk', 60.0, 68),
 ('True Taste, 2% Milk', 60.0, 934),
 ('TruTaste Lactose Free 2% Milk', 50.0, 282),
 ('Organic Partly Skimmed 2% Milk', 50.0, 321),
 ('Organic Partly Skimmed 2% Milk', 50.0, 322),
 ('1% Milk', 50.0, 473),
 ('Trutaste Lactose Free 2% Milk', 50.0, 569),
 ('Trutaste Lactose Free 2% Milk', 50.0, 612),
 ('2% Lactose Free Enriched Milk', 50.0, 662),
 ('2% Lactose Free Chocolate Enriched Milk', 42.85714285714286, 245),
 ('2% M.F. Partly Skimmed Milk', 42.85714285714286, 953),
 ('Flavoured Milk, Chocolate', 40.0, 92),
 ('TruTaste 1% Milk', 40.0, 259),
 ('Flavoured Milk, Strawberry', 40.0, 351),
 ('Yogurt, Plain 2%', 40.0, 385),
 ('Flavoured Milk, Chocolate', 40.0, 475),
 ('TruTaste 1% Milk', 40.0, 614),
 ('TruTaste 1% Milk', 40.0, 708),
 ('Flavoured Milk, Chocolate', 40.0, 859),
 ('Milk, Ultr

In [107]:
from itertools import combinations

def item_selection(df1, df2):
    df = df1.append(df2, ignore_index=True)
    dfc = df.groupby('list_item')['comparable_price']
    df = df.assign(min_cost=dfc.transform(min))
    df = df[df['comparable_price'] == df['min_cost']]

    per_unit_subtotal = sum(df['comparable_price'])

    return df, per_unit_subtotal

results = {}

def n_store_selection(n, results_dict):
    # if n = 1:

    possibilities = list(combinations(results_dict.keys(), n))
    for combin in possibilities:
        df1 = pd.read_csv(f'search_output/{combin[0]}_results.csv')
        df2 = pd.read_csv(f'search_output/{combin[1]}_results.csv')

        optimal_selection, per_unit_subtotal = item_selection(df1, df2)

        results[combin] = per_unit_subtotal

    return results

n_store_selection(3, results_dict)

{('zehrs', 'no_frills', 'valu_mart'): 4.36,
 ('zehrs', 'no_frills', 'sobeys'): 6.603324443831867,
 ('zehrs', 'no_frills', 'freshco'): 6.0631486721469505,
 ('zehrs', 'valu_mart', 'sobeys'): 6.695622226465402,
 ('zehrs', 'valu_mart', 'freshco'): 5.738218348093484,
 ('zehrs', 'sobeys', 'freshco'): 5.9037967803848135,
 ('no_frills', 'valu_mart', 'sobeys'): 6.695622226465402,
 ('no_frills', 'valu_mart', 'freshco'): 5.738218348093484,
 ('no_frills', 'sobeys', 'freshco'): 5.9037967803848135,
 ('valu_mart', 'sobeys', 'freshco'): 5.9037967803848135}

In [112]:
d = [1,2,3]
print(d[0])
d.remove(d[0])
d

1


[2, 3]

In [117]:
from itertools import combinations

def item_selection(dfs):
    df = dfs[0]
    dfs.remove(dfs[0])
    for df_n in dfs:
        df = df.append(df_n, ignore_index=True)
    # df = df1.append(df2, ignore_index=True)
    df_grp = df.groupby('list_item')['comparable_price']
    df = df.assign(min_cost=df_grp.transform(min))
    df = df[df['comparable_price'] == df['min_cost']]

    per_unit_subtotal = sum(df['comparable_price'])

    return df, per_unit_subtotal

results = {}

def n_store_selection(n, results_dict):
    # if n = 1:

    possibilities = list(combinations(results_dict.keys(), n))
    for combin in possibilities:
        dfs = []
        for store in combin:
            df = pd.read_csv(f'search_output/{store}_results.csv')
            dfs.append(df)

        optimal_selection, per_unit_subtotal = item_selection(dfs)

        results[combin] = per_unit_subtotal

    return results

n_store_selection(1, results_dict)

{('zehrs',): 6.199999999999999,
 ('no_frills',): 4.53,
 ('valu_mart',): 18.11,
 ('sobeys',): 7.2838562670982165,
 ('freshco',): 5.9037967803848135}

In [118]:
len(results_dict)

5

In [2]:
import pandas as pd

In [9]:
df = pd.read_csv('search_output/zehrs_results.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,store,category,brand,product,price,sale_price,per_unit_price,sale_per_unit_price,units,is_sale,list_item,similarity,comparable_PUP,comparable_price
0,zehrs,dairy_and_eggs,Neilson,2% Milk,3.39,NaN,0.34,NaN,100ml,0.0,2% milk,100.000000,0.34,3.39
1,zehrs,dairy_and_eggs,No Name,Medium Cheddar Cheese,4.29,NaN,2.15,NaN,100g,0.0,Cheddar Cheese,66.666667,2.15,4.29
2,zehrs,bakery,Wonder,"Bread, Sliced White",3.79,NaN,0.56,NaN,100g,0.0,white sliced bread,75.000000,0.56,3.79


In [13]:
df.to_dict('list')

{'store': ['zehrs', 'zehrs', 'zehrs'],
 'category': ['dairy_and_eggs', 'dairy_and_eggs', 'bakery'],
 'brand': ['Neilson', 'No Name', 'Wonder'],
 'product': ['2% Milk', 'Medium Cheddar Cheese', 'Bread, Sliced White'],
 'price': [3.39, 4.29, 3.79],
 'sale_price': [nan, nan, nan],
 'per_unit_price': [0.34, 2.15, 0.56],
 'sale_per_unit_price': [nan, nan, nan],
 'units': ['100ml', '100g', '100g'],
 'is_sale': [0.0, 0.0, 0.0],
 'list_item': ['2% milk', 'Cheddar Cheese', 'white sliced bread'],
 'similarity': [100.0, 66.66666666666667, 75.0],
 'comparable_PUP': [0.34, 2.15, 0.56],
 'comparable_price': [3.39, 4.29, 3.79]}

In [10]:
import pandas as pd

stores = ['walmart', 'freshco', 'sobeys', 'food_basics']

store = 'freshco'
# for store in stores:
flipp = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')
synth = pd.read_csv(f'clean_data/{store}/synthetic_data.csv')

all_data = flipp.append(synth)


all_data.head()
# all_data.to_csv(f'clean_data/{store}/{store}_data.csv', index=False)

In [ ]:
all_data.tail()